# Set up Account APIs
Running this script will create the APIs that we will be monitoring for accounts and transactions. They require user authentication. 

The API setups are defined with tags that allow us to easily provide OBIE metric reporting.

## Requirements
1. Create an APImetrics Project
1. Get an API key with EDITOR permissions for the project, enter it below
1. Get the maTLS transport certificate and key in PEM format, enter paths below
1. Run this workbook to create all APIs
1. Create an environment variable "financial_id" - TBD - add this here

NB: Until we publish a Conditions API, you will need to add conditions to the API to extract variables

In [ ]:
# Enter intended project's APImetrics API key here:
API_KEY = input("Enter your APImetrics API key for the intended project")

In [ ]:
# Enter the bank's full path, including https and without trailing /
BANK_URL = input("Enter the full URL path for the AISP APIs without trailing / :") or 'https://example.com/open-banking/3.1/aisp'

SSL_KEY_PATH = "" #"./local/TPP_OB_Transport.key"
SSL_CERT_PATH = "" #"./local/TTP_OB_Transport.pem"

OAUTH_METHOD = "private_key_jwt"
# if OAUTH_METHOD == "client_secret_post" or OAUTH_METHOD == "client_secret_basic":
# # CLIENT_ID = '...'
# # CLIENT_SECRET = '...'

# The OBIE version ID we're working against
VERSION = "3.1"

from urllib.parse import urlparse

url_info = urlparse(BANK_URL)
DOMAIN, _, _ = url_info.netloc.partition(':') # removes the port 

In [ ]:
# Helper functions
import requests
import json
from apimetrics_api import APImetricsAPI 

# An instance of the class that calls the APImetrics API
CLIENT = APImetricsAPI(API_KEY)

# [CLIENT.delete_token(o['id']) for o in CLIENT.tokens.values()]
# [CLIENT.delete_auth(o['id']) for o in CLIENT.auths.values()]
# [CLIENT.delete_call(o['id']) for o in CLIENT.calls.values()]
# [CLIENT.delete_workflow(o['id']) for o in CLIENT.workflows.values()]

In [ ]:

# All the account APIs share a similar setup, this is a mapping of
# our tag to the OBIE tag, the path and the friendly name
SETUP_MAP = {
    "bulk:accounts": (4, "/accounts", "Accounts: Bulk"),
    "bulk:balances": (7, "/balances", "Balances: Bulk"),
    "bulk:transactions": (9, "/transactions", "Transactions: Bulk"),
    "bulk:beneficiaries": (11, "/beneficiaries", "Beneficiaries: Bulk"),
    "bulk:directdebits": (13, "/direct-debits", "Direct-Debits: Bulk"),
    "bulk:standingorders": (15, "/standing-orders", "Standing-Orders: Bulk"),
    "bulk:products": (17, "/products", "Products: Bulk"),
    "bulk:offers": (19, "/offers", "Offers: Bulk"),
    "bulk:party": (21, "/party", "Party: Bulk"),
    "bulk:scheduledpayments": (23, "/scheduled-payments", "Scheduled-Payments: Bulk"),
    "bulk:statements": (28, "/statements", "Statements: Bulk"),
    "account:account": (5, "/accounts/__ACCOUNT_ID__", "Accounts: Get for AccountId"),
    "account:balance": (6, "/accounts/__ACCOUNT_ID__/balances", "Balances: Get for AccountId"),
    "account:transactions": (8, "/accounts/__ACCOUNT_ID__/transactions", "Transactions: Get for AccountId"),
    "account:beneficiaries": (10, "/accounts/__ACCOUNT_ID__/beneficiaries", "Beneficiaries: Get for AccountId"),
    "account:directdebits": (12, "/accounts/__ACCOUNT_ID__/direct-debits", "Direct-Debits: Get for AccountId"),
    "account:standingorders": (14, "/accounts/__ACCOUNT_ID__/standing-orders", "Standing-Orders: Get for AccountId"),
    "account:product": (16, "/accounts/__ACCOUNT_ID__/product", "Product: Get for AccountId"),
    "account:offers": (18, "/accounts/__ACCOUNT_ID__/offers", "Offers: Get for AccountId"),
    "account:party": (20, "/accounts/__ACCOUNT_ID__/party", "Party: Get for AccountId"),
    "account:scheduledpayments": (22, "/accounts/__ACCOUNT_ID__/scheduled-payments", "Scheduled-Payments: Get for AccountId"),
    "account:statements": (25, "/accounts/__ACCOUNT_ID__/statements/__STATEMENT_ID__", "Statements: Get for StatementId"),
    "account:statement": (24, "/accounts/__ACCOUNT_ID__/statements", "Statements: Get for AccountId"),
    "account:statement_file": (26, "/accounts/__ACCOUNT_ID__/statements/__STATEMENT_ID__/file", "Statements: File for StatementId"),
    "account:statement_transactions": (27, "/accounts/__ACCOUNT_ID__/statements/__STATEMENT_ID__/transactions", "Statements: Transactions for StatementId"),
}

# Helper function - the APImetrics API setup object
def get_call(ob_id, path, name, tag):
    setup = {
        "meta": {
            "description": "Get a list of accounts",
            "tags": [
                "api_type:read",
                "sector:financial",
                "ob_id:{}".format(ob_id),
                "ob_v:{}".format(VERSION),
                tag,
            ],
            "name": "v{}: {}".format(VERSION, name),
            "workspace": "global",
        },
        "request": {
            "body": None,
            "parameters": [],
            "url": "{}{}".format(BANK_URL, path),
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": [
                {"key": "Accept", "value": "application/json"},
                {"key": "x-fapi-financial-id", "value": "{{financial_id}}"},
            ],
            "token_id": CLIENT.tokens_by_auth[CLIENT.auths_by_tag['auth:bank_matls']],
            "method": "GET",
        },
    }
    return setup

def get_call_setup(call_tag):
    ob_id, path, name = SETUP_MAP[call_tag]
    return get_call(ob_id, path, name, f"banks:{VERSION}:{call_tag}")

def create_call_if_needed(tag):
    if f"banks:{VERSION}:{tag}" not in CLIENT.calls_by_tag:
        setup = get_call_setup(tag)
        data = CLIENT.create_call(setup)
        print(f"Created API {data['meta']['name']} with id {data['id']}")
        return data['id']

## Create Auth Setting and Token for Bank

In [ ]:
# # First create Auth Setting
# tag = 'auth:bank_matls'
# if tag not in CLIENT.auths_by_tag:

#     ssl_key = None
#     ssl_cert = None

#     if SSL_KEY_PATH:
#         with open(SSL_KEY_PATH) as stream:
#             ssl_key = stream.read()

#     if SSL_CERT_PATH:
#         with open(SSL_CERT_PATH) as stream:
#             ssl_cert = stream.read()
    
#     setup = {
#         "keys": {},
#         "meta": {
#             "domain": DOMAIN,
#             "documentation": {"keys": "", "docs": "", "apps": "", "provider": ""},
#             "name": "Transport MATLS",
#             "tags": [tag],
#             "description": "Mutual Authenticated TLS for calls to bank APIs",
#         },
#         "settings": {
#             "auth_type": "MANUAL",
#             "ssl_key": ssl_key,
#             "ssl_cert": ssl_cert,
#         },
#     }
#     if OAUTH_METHOD == "client_secret_post" or OAUTH_METHOD == "client_secret_basic":
#         setup["keys"]["client_id"] = CLIENT_ID
#         setup["keys"]["client_secret"] = CLIENT_SECRET
    
#     auth = CLIENT.create_auth(setup)
#     print(f"Created Auth Setting {auth['meta']['name']} with id {auth['id']}")
    
# # Second Create Token
# if CLIENT.auths_by_tag['auth:bank_matls'] not in CLIENT.tokens_by_auth:
#     setup = {
#         'meta': {
#             'name': 'Authenticated User Access Token',
#             'domain': DOMAIN,
#             'auth_id': CLIENT.auths_by_tag['auth:bank_matls']
#         },
#         'token': {}
#     }
#     token = CLIENT.create_token(setup)
#     print(f"Created Auth Token {token['meta']['name']} with id {token['id']}")


## Create APIs

### Bulk APIs

In [ ]:
CLIENT.auths_by_tag['auth:bank_matls'] = "agpzfnZpYXRlc3RzchoLEg1TZXJ2aWNlQ29uZmlnGICAmMiwnOEIDA"
tag = "bulk:accounts"
id_str = create_call_if_needed(tag)
conditions = {
    "conditions": [
        {
          "source": "RESPONSE_BODY", 
          "test_result_on_true": None, 
          "val": "", 
          "variable_path": "Data.Account[0].AccountId", 
          "variable_name": "ACCOUNT_ID", 
          "test_result_on_false": "CONTENT_ERROR", 
          "condition": "EXISTS"
        },
      ]
}
CLIENT.set_call_conditions(id_str, conditions)

In [ ]:
tag = "bulk:balances"
create_call_if_needed(tag)

In [ ]:
tag = "bulk:transactions"
create_call_if_needed(tag)

In [ ]:
tag = "bulk:beneficiaries"
create_call_if_needed(tag)

In [ ]:
tag = "bulk:directdebits"
create_call_if_needed(tag)

In [ ]:
tag = "bulk:standingorders"
create_call_if_needed(tag)

In [ ]:
tag = "bulk:products"
create_call_if_needed(tag)

In [ ]:
tag = "bulk:offers"
create_call_if_needed(tag)

In [ ]:
tag = "bulk:party"
create_call_if_needed(tag)

In [ ]:
tag = "bulk:scheduledpayments"
create_call_if_needed(tag)

In [ ]:
tag = "bulk:statements"
id_str = create_call_if_needed(tag)
conditions = {
    "conditions": [
        {
          "source": "RESPONSE_BODY", 
          "test_result_on_true": None, 
          "val": "", 
          "variable_path": "Data.Statement[0].StatementId", 
          "variable_name": "STATEMENT_ID", 
          "test_result_on_false": "CONTENT_ERROR", 
          "condition": "EXISTS"
        },
      ]
}
CLIENT.set_call_conditions(id_str, conditions)

### Account APIs

In [ ]:
tag = "account:account"
create_call_if_needed(tag)

In [ ]:
tag = "account:balance"
create_call_if_needed(tag)

In [ ]:
tag = "account:transactions"
create_call_if_needed(tag)

In [ ]:
tag = "account:beneficiaries"
create_call_if_needed(tag)

In [ ]:
tag = "account:directdebits"
create_call_if_needed(tag)

In [ ]:
tag = "account:standingorders"
create_call_if_needed(tag)

In [ ]:
tag = "account:product"
create_call_if_needed(tag)

In [ ]:
tag = "account:offers"
create_call_if_needed(tag)

In [ ]:
tag = "account:party"
create_call_if_needed(tag)

In [ ]:
tag = "account:scheduledpayments"
create_call_if_needed(tag)

In [ ]:
tag = "account:statements"
id_str = create_call_if_needed(tag)
conditions = {
    "conditions": [
        {
          "source": "RESPONSE_BODY", 
          "test_result_on_true": None, 
          "val": "", 
          "variable_path": "Data.Statement[0].StatementId", 
          "variable_name": "STATEMENT_ID", 
          "test_result_on_false": "CONTENT_ERROR", 
          "condition": "EXISTS"
        },
      ]
}
CLIENT.set_call_conditions(id_str, conditions)

In [ ]:
tag = "account:statement"
create_call_if_needed(tag)

In [ ]:
tag = "account:statement_file"
create_call_if_needed(tag)

In [ ]:
tag = "account:statement_transactions"
create_call_if_needed(tag)

## Workflow for Account APIs

In [ ]:
tag = 'banks:3.1:account:apis'
if tag not in CLIENT.workflows_by_tag:
    call_tags = [
        "banks:3.1:bulk:accounts",    
        "banks:3.1:account:account",
        "banks:3.1:account:balance",
        "banks:3.1:account:transactions",
        "banks:3.1:account:beneficiaries",
        "banks:3.1:account:directdebits",
        "banks:3.1:account:standingorders",
        "banks:3.1:account:product",
        "banks:3.1:account:offers",
        "banks:3.1:account:party",
        "banks:3.1:account:scheduledpayments",
        "banks:3.1:account:statements",
        "banks:3.1:account:statement",
        "banks:3.1:account:statement_file",
        "banks:3.1:account:statement_transactions",
    ]
    for t in call_tags:
        assert t in CLIENT.calls_by_tag, f"API {t} does not exist"

    setup = {
      "meta": {
        "name": "v3.1: Account APIs", 
        "workspace": "global", 
        "tags": [tag], 
      }, 
      "workflow": {
        "handle_cookies": False,
        "stop_on_failure": False,
        "call_ids": [CLIENT.calls_by_tag[t] for t in call_tags]
      }
    }
    workflow = CLIENT.create_workflow(setup)
    print(f"Created Workflow {workflow['meta']['name']} with id {workflow['id']}")